# Introduction to Python and Jupyter Notebooks

## Data Manipulation using Pandas

This is the reference component to the NMSU Pandas workshop 2018.

## Set Up
Run the following cell to load (1) the pandas library and (2) the dataset we will be working with today `(Hint: run the code by pushing shift + enter)`

In [1]:
import pandas as pd
pols = pd.read_csv("pollutants.csv", dtype = object)

Selecting specific values of a `pandas` `DataFrame` or `Series` to work on is an implicit step in almost any data operation you'll run. Hence a solid understanding of how to slice and dice a dataset is vital.

We specified dtype = object earlier because there are many different data types present in this data frame- yet all of them our objects.  Now, we need to convert some columns to numeric for manipulation with the numpy library (Pandas uses numpy, numpy uses numbers)

In [ ]:
cols = ['fiscal_year', 'result']
pols[cols] = pols[cols].apply(pd.to_numeric, errors = 'coerce')

## Native accessors

Native Python objects provide many good ways of indexing data. `pandas` carries all of these over, which helps make it easy to start with.

Consider this `DataFrame` (Hint: run the code by pushing shift + enter)

In [ ]:
pols

In Python we can access the property of an object by accessing it as an attribute. A `book` object, for example, might have a `title` property, which we can access by calling `book.title`. Columns in a `pandas` `DataFrame` work in much the same way. 

Hence to access the `fiscal_year` property of our `pols` we can use:

In [ ]:
pols.fiscal_year

If we have a `dict` object in Python, we can access its values using the indexing (`[]`) operator. Again, we can do the same with `pandas` `DataFrame` columns. It "just works":

In [ ]:
pols['fiscal_year']

These are the two ways of selecting a specific columnar `Series` out of a `pandas` `DataFrame`. Neither of them is more or less syntactically valid than the other, but the indexing operator `[]` does have the advantage that it can handle column names with reserved characters in them (e.g. if we had a column named `fiscal year` instead of `fiscal_year`, `pols.fiscal year` wouldn't work). For the same reason, people tend to name objects with underscores instead of spaces

Doesn't a `pandas` `Series` look kind of like a fancy `dict`? It pretty much is, so it's no surprise that, to drill down to a single specific value, we need only use the indexing operator `[]` once more:

In [ ]:
pols['fiscal_year'][0]

## Index-based selection

The indexing operator and attribute selection are nice because they work just like they do in the rest of the Python ecosystem. As a novice, this makes them easy to pick up and use. However, `pandas` has its own accessor operators, `loc` and `iloc`. For more advanced operations, these are the ones you're supposed to be using.

`pandas` indexing works in one of two paradigms. The first is **index-based selection**: selecting data based on its numerical position in the data. `iloc` follows this paradigm.

To select the first row of data in this `DataFrame`, we may use the following:

In [ ]:
pols.iloc[0]

Both `loc` and `iloc` are row-first, column-second. This is the opposite of what we do in native Python, which is column-first, row-second.

This means that it's marginally easier to retrieve rows, and marginally harder to get retrieve columns. To get a column with `iloc`, we can do the following (`We're selecting 3 to get the general_location column- Pandas indexing starts with 0 not 1`):

In [ ]:
pols.iloc[:, 3]

On its own the `:` operator, which also comes from native Python, means "everything". When combined with other selectors, however, it can be used to indicate a range of values. For example, to select the `country` column from just the first, second, and third row, we would do:

In [ ]:
pols.iloc[:3, 3]

Or, to select just the second and third entries, we would do:

In [ ]:
pols.iloc[1:3, 3]

It's also possible to pass a list:

In [ ]:
pols.iloc[[0, 1, 2], 3]

Finally, it's worth knowing that negative numbers can be used in selection. This will start counting forwards from the _end_ of the values. So for example here are the last five elements of the dataset.

In [ ]:
pols.iloc[-5:]

## Label-based selection

The second paradigm for attribute selection is the one followed by the `loc` operator: **label-based selection**. In this paradigm it's the data index value, not its position, which matters.

For example, to get the first entry in `pols`, we would now do the following:

In [ ]:
pols.loc[0, 'fiscal_year']

`iloc` is conceptually simpler than `loc` because it ignores the dataset's indices. When we use `iloc` we treat the dataset like a big matrix (a list of lists), one that we have to index into by position. `loc`, by contrast, uses the information in the indices to do its work. Since your dataset usually has meaningful indices, it's usually easier to do things using `loc` instead. For example, here's one operation that's much easier using `loc`:

In [ ]:
pols.loc[:, ['fiscal_year', 'parameter', 'result', 'units']]

When choosing or transitioning between `loc` and `iloc`, there is one "gotcha" worth keeping in mind, which is that the two methods use slightly different indexing schemes.

`iloc` uses the Python stdlib indexing scheme, where the first element of the range is included and the last one excluded. So 0:10 will select entries 0,...,9. `loc`, meanwhile, indexes inclusively. So 0:10 will select entries 0,...,10.

Why the change? Remember that loc can index any stdlib type: strings, for example. If we have a DataFrame with index values `Apples, ..., Potatoes, ...`, and we want to select "all the alphabetical fruit choices between Apples and Potatoes", then it's a heck of a lot more convenient to index `df.loc['Apples':'Potatoes']` than it is to index something like `df.loc['Apples', 'Potatoet]` (`t` coming after `s` in the alphabet).

This is particularly confusing when the `DataFrame` index is a simple numerical list, e.g. `0,...,1000`. In this case `df.iloc[0:1000]` will return 1000 entries, while `df.loc[0:1000]` return 1001 of them! To get 1000 elements using `loc`, you will need to go one lower and ask for `df.iloc[0:999]`. Earlier versions of this tutorial did not point this out explicitly, leading to a lot of user confusion on some of the related answers, so we've included this note here explaining this issue.

Otherwise, the semantics of using `loc` are the same as those for `iloc`.

## Manipulating the index

Label-based selection derives its power from the labels in the index. Critically, the index we use is not immutable. We can manipulate the index in any way we see fit.

The `set_index` method can be used to do the job. Here is what happens when we `set_index` to the `result` field:

In [ ]:
pols.set_index("result", inplace = True)
pols

This index, however, is not so informative. If you're going to label the rows of your DataFrame, it would be good to label them in a manner which is more meaningful than the current one. Let's restore the original index

In [ ]:
pols.reset_index(inplace = True)
pols

## Data Evaluation

So far we've been indexing various strides of data, using structural properties of the `DataFrame` itself. To do *interesting* things with the data, however, we want to find some `answers` 

Usually I begin by finding as much information about the data as I can. For example, what are the different general locations that are present in the dataset? This requires the numpy function `unique()`

In [ ]:
pols.general_location.unique()

Well, that's quite a list!  How many different places, exactly? This requires the built-in python function `len()`

In [ ]:
len(pols.general_location.unique())

## Dealing with missing data

To perform numerical assessments or manipulations, we must account for missing data.  This can be done in a variety of ways.  The simplest, perhaps, is just to drop the rows for which the value is NaN. 

Let's find the range of fiscal years in this data set. To do this, we must first drop the NA values like this:

In [ ]:
pols.dropna(subset = ['fiscal_year'], inplace = True)
pols

Next, we use the built-in python functions min() and max() to evaluate the date range as follows:

In [ ]:
pols.fiscal_year.min()

In [ ]:
pols.fiscal_year.max()

We could also sort the dataframe by year, then find the first and the last entry as follows:

In [ ]:
result = pols.sort_values('fiscal_year')
print(result.fiscal_year.head(1))
print(result.fiscal_year.tail(1))


## Conditional selection

So far we've been indexing various strides of data, using structural properties of the `DataFrame` itself. To do *interesting* things with the data, however, we often need to ask questions based on conditions. 

For example, suppose we're interested in finding out ways that pollutants change over time. We don't know how many locations have a range of dates, but we can start by asking where the first tests were done like this:


In [ ]:
pols.fiscal_year == 1986

This operation produced a `Series` of `True`/`False` booleans based on the `fiscal_year` of each record.  This result can then be used inside of `loc` to select the relevant data:

In [ ]:
pols.loc[pols.fiscal_year == 1986]

We can see all of the locations that were tested in 1986 like this:

In [ ]:
pols.loc[pols.fiscal_year == 1986].general_location.unique()

Similarly, we can view the test locations ten years later:

In [ ]:
pols.loc[pols.fiscal_year == 1996].general_location.unique()

In [ ]:
pols

If we look, we can see that Tampa Bay is in both of those lists, so we can view how pollutants in Tampa Bay change over time. First we will create a dataframe containing just the Tampa Bay tests like this:

In [ ]:
tb = pols.loc[pols.general_location == 'Tampa Bay']
tb

 
Next we can sort by `result` which corresponds to pollutant concentration and then group by `parameter` like this:

In [ ]:
tb.sort_values('result', ascending = False)

In [ ]:
top5 = tb.groupby(['parameter', 'units']).head(5)
top5

Well, that isn't very interesting.  Let's drop the zeros and try again. We create a new dataframe for tampa bay based on these criteria using `&` as follows:

In [ ]:
tb3 = pols.loc[(pols.general_location == 'Tampa Bay') & (pols.result > 0)]
tb3

Let's slice out the columns we are interested in like this: 

In [ ]:
tb3.iloc[0]

In [ ]:
tb4 = tb3.iloc[: , [0,3,4,11,21]]
tb4

Then we sort and group as before

In [ ]:
tb4.sort_values('result', ascending = False)

In [ ]:
top5 = tb4.groupby(['parameter', 'units']).head(5)
top5

This is still not very clear.  It's time to introduce a very fun pandas function- `pivot_table`- which is a fun way to get an overview of your data really quickly. Let's see which pollutants changed the most through the years of the tampa bay study (`the default function in pivot_table is the mean but we could also aggregate the results by other functions`): 

In [ ]:

tb4.pivot_table(values = 'result', columns = 'fiscal_year', index = 'parameter')

Well, that was sort of fun...  but I want to be able to work with the data not just look at it.  Let's turn the pivot table into a dataframe:

In [ ]:
df = pd.DataFrame(tb4.pivot_table(values = 'result', columns = 'fiscal_year', index = 'parameter' ))
df

Let's find out which pollutants changed the most in the Tampa Bay data

In [ ]:
## we first append the dataframe
## with the minimim and maximum across all the columns
df['small'] = df.min(axis = 1)
df['large'] = df.max(axis = 1)
## axis = 1 means across columns
## the default is axis = 0
df.info()
## df.info tells you about the DataFrame

now we can take the difference between the smallest and largest values

In [ ]:
df['change'] = df.large - df.small
df.info()

In [ ]:
df['change'] = df['change'].apply(pd.to_numeric, errors = 'coerce')
df

In [ ]:
df2 = df.sort_values('change', ascending = False)
df2

Suppose we wanted to look at specific types of pollutants- for example, all of the PCBs.  We create a new dataframe containing these columns. First we get rid of the missing data here

In [ ]:
pols.dropna(subset = ['parameter'], inplace = True)

Now we search for parameters that contain the letters `PCB`, and we omit the zero values.

In [ ]:
pcbs= pols.loc[pols['parameter'].str.contains("PCB") & (pols.result > 0)]
pcbs

Now we can do a pivot table `(stored as a data frame)` to evaluate individual PCBs and the test years, to start to look at changes over time

In [ ]:
pcbDF = pd.DataFrame(pcbs.pivot_table(values = 'result', columns = 'parameter', index = 'fiscal_year' ))
pcbDF

let's see when the cummulative PCB values rise and fall. First we take the sum across the dataFrame Columns like this: 

In [ ]:
pcbDF['all'] = pcbDF.sum(axis = 1)
pcbDF

Next, we sort by the `sum` value that we just stored in the `all` column

In [ ]:
pcbDF.sort_values('all', ascending = False)

Let's try to determine WHY 2004 (the most PCBs) was more polluted than the previous decade. Were more tests done?  Different types of tests?  Different areas tested? We will go back to our pcb dataframe to look at this information more closely.

In [ ]:
oh494 = pcbs.loc[(pcbs.fiscal_year == 2004) | (pcbs.fiscal_year == 1994)]
## the pipe operater denotes "or"
oh494 



How many tests were from 2004?

In [ ]:
len(oh494.loc[oh494.fiscal_year==2004])

How many tests were from 1994?

In [ ]:
len(oh494.loc[oh494.fiscal_year==1994])

That's quite a difference- more than twice the number of tests  What locations were tested in each of those years?

In [ ]:
oh494[oh494.fiscal_year==2004].general_location.unique()

In [ ]:
oh494[oh494.fiscal_year==1994].general_location.unique()

In [ ]:
Lake Michigan was tested in both of those years, lets evaluate PCBs there in more detail

In [ ]:
LM = oh494.loc[oh494.general_location == 'Lake Michigan']
LM

Let's do another pivot table to look at the PCB changes in Lake Michigan

In [ ]:
LM.iloc[0]

In [ ]:
LM2 = LM.pivot_table(values = 'result', columns = 'fiscal_year', index = 'parameter' )
LM2

which PCB's changed the most in Lake Michigan during that decade?

In [ ]:
LM2.diff(axis = 1)

well, that's great, but how do I work with that data? Let's make a new DataFrame

In [ ]:
LMfnl = LM2.diff(axis = 1)
LMfnl

is this a dataframe?  Let's find out

In [ ]:
LMfnl.info()

Now, let's see which PCBs are responsible for the biggest changes

In [ ]:
LMfnl.sort_values(2004.0)

well, we have a conclusion: PCB101_90 decreased from 1994 to 2004, while PCB153_132_168 is responsible for the biggest increase. 

### I hope you enjoyed reading- come back if you need help with the companion exercises.  Everything you need to know is right here!